<a href="https://colab.research.google.com/github/Nilanshrajput/AC2MC-simulator/blob/master/Model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:

!pip install pytorch-lightning
!pip install wandb

In [1]:
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

import os
from argparse import Namespace
from typing import *
import random
from functools import lru_cache

import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.metrics import Accuracy

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torch.nn as nn
import torch
from collections import OrderedDict 
from sklearn.preprocessing import LabelEncoder

In [3]:

inventory = ['Deagle', 'Elite', 'FiveSeven', 'Glock', 'Ak47', 'Aug', 'Awp', 'Famas', 'G3sg1', 'GalilAr', 'M249', 'M4a4', 'Mac10', 'P90', 'Mp5sd', 'Ump45', 'Xm1014', 'Bizon', 'Mag7', 'Negev', 'Sawedoff', 'Tec9', 'ZeusX27', 'P2000', 'Mp7', 'Mp9', 'Nova', 'P250', 'Scar20', 'Sg553', 'Ssg08', 'M4a1S', 'UspS', 'Cz75Auto', 'R8Revolver', 'Flashbang', 'HeGrenade', 'SmokeGrenade', 'MolotovGrenade', 'DecoyGrenade', 'IncendiaryGrenade', 'Knife', 'C4']
maps = ['de_dust2', 'de_mirage', 'de_nuke', 'de_inferno', 'de_overpass',
       'de_vertigo', 'de_train', 'de_cache']

In [4]:
maps_label = {map:i+1 for i,map in enumerate(maps)}
round_status = {'BombPlanted':1, 'FreezeTime':2, 'Normal':3, "SlackTime":4}

In [5]:
bins = {
    'round_time_left':[0, 10, 30, 50, 70, 100,115,4000],
    'health':[0,10,20,30,40,50,65,80,90,100],
    'money':[0,500,100.,1500,2200,3000,3500,3800,4000,4500,5000,5500,7000,9000,12000,16000],
    'clip' : [0,1,2,10,20,30,35,300],
    'ammo' :[0,2,30,60,120,300],
    'smokes':[0,1,2,3,4,10],
    'molotov':[0,1,2,3,4,10],
    'armor':[0,10,40,80,100]
}



In [6]:

class CsData(Dataset):
    def __init__(self, dfx,dfy):
        self.dfx = dfx
        self.dfy = dfy
        self.dfx['map'] = self.dfx['map'].apply(lambda x: maps_label[x])
        self.dfx['round_status'] = self.dfx['round_status'].apply(lambda x: round_status[x])
        self.dfx['round_status_time_left_binned']=self.dfx['round_status_time_left'].apply(lambda x: self.encode_time(x))
        

    def __getitem__(self, id):
        row = self.dfx.iloc[id]
        outs = self.process_row(row)
        #outs = tuple(outs.values())
        if len(outs)!= 236:
            print(id)
        return tuple(outs), torch.FloatTensor([self.dfy.iloc[id]])

    def __len__(self):
        return len(self.dfx)

    def process_row(self, row):
        
        row_embs = []#OrderedDict()

        #map
        row_embs.append(torch.LongTensor([row['map']]))

        #cts and ts
        cts = [p for p in row['alive_players'] if p['team']=='CT']
        ts = [p for p in row['alive_players'] if p['team']=='Terrorist']
        row_embs.extend(self.encode_alive_players(cts))
        row_embs.extend(self.encode_alive_players(ts))

        # GRS
        
        grs = OrderedDict()
        grs['molotovs'] = self.label_molotovs(len(row['active_molotovs']))
        grs['smokes'] = self.label_smokes(len(row['active_smokes']))
        row_embs.extend(grs.values())
        
        #round
        round = OrderedDict()
        round['round_status'] = torch.LongTensor([row['round_status']])
        round['round_status_time_left_binned']= torch.LongTensor([row['round_status_time_left_binned']])
        row_embs.extend(round.values())

        #score
        row_embs.append(self.encode_score(row['current_score']))


        return row_embs

    def encode_time(self, time):
        bins = [0, 10, 30, 50, 70, 100,115,4000]
        return (np.digitize([time],bins)[0])+1

    def encode_alive_players(self,alive_pls):
        encoding = []
        for i,plr in enumerate(alive_pls):
            encoding.extend(self.player_encoder(plr))
            # bad dta contains more thn 5 in a team
            if i==4:
                break

        for _ in range(len(alive_pls),5):
            encoding.extend(self.player_encoder(pad=True))

        return encoding

    def player_encoder(self, plr=None, pad=False):
        embs =[]# OrderedDict()
        if not pad:
            health = self.label_health(plr['health'])
            embs.append( health)
            #embs.append(self.embeddings['health_bin'](health)[0])
        
            armor= self.label_armor(plr['armor'])
            embs.append(armor)
            #embs.append(self.embeddings['armor'](armor)[0])
                        
            money = self.label_money(plr['money'])
            embs.append(money)
            #embs.append(self.embeddings['money'](money)[0] )
                        
            has_defuser = self.label_has_defuser(plr['has_defuser'])
            embs.append(has_defuser)
            #embs.append(self.embeddings['has_defuser'](has_defuser)[0])
                        
            has_helmet = self.label_has_helmet(plr['has_helmet'])
            embs.append(has_helmet)
            #embs.append(self.embeddings['has_helmet'](has_helmet)[0])

            #embs['inventory']=[]
            for i,item in enumerate(plr['inventory']):
                embs.extend(self.encode_inventory(item).values())
                # max 6 items considered
                if i==5:
                    break
            for _ in range(len(plr['inventory']),6):
                embs.extend(self.encode_inventory(pad=True).values())

        else:

            health = torch.LongTensor([0])
            embs.append(health)
            #embs.append(self.embeddings['health_bin'](health)[0])
        
            armor= torch.LongTensor([0])
            embs.append(armor)
            #embs.append(self.embeddings['armor'](armor)[0])
                        
            money = torch.LongTensor([0])
            embs.append(money)
            #embs.append(self.embeddings['money'](money)[0] )
                        
            has_defuser = torch.LongTensor([0])
            embs.append(has_defuser)
            #embs.append(self.embeddings['has_defuser'](has_defuser)[0])
                        
            has_helmet = torch.LongTensor([0])
            embs.append(has_helmet)
            #embs.append(self.embeddings['has_helmet'](has_helmet)[0])

            #embs['inventory']=[]
            for _ in range(6):
                embs.extend(self.encode_inventory(pad=True).values())

        return embs


    def encode_inventory(self, item=None, pad = False):
        embs = OrderedDict()
        if not pad:
            typ = item['item_type']
            typ = torch.LongTensor([inventory.index(typ)])
            embs['item'] = typ
            #embs.append(self.embeddings['item'](typ)[0])

            clip = self.label_clip(item['clip_ammo'])
            embs['clip'] = clip
            #embs.append(self.embeddings['clip'](clip)[0])

            ammo = self.label_ammo(item['clip_ammo'])
            embs['ammo'] = ammo
            #embs.append(self.embeddings['ammo'](ammo)[0])

        else: 
            typ = torch.LongTensor([0])
            embs['item'] = typ
            #embs.append(self.embeddings['item'](typ)[0])

            clip = torch.LongTensor([0])
            embs['clip'] = clip
            #embs.append(self.embeddings['clip'](clip)[0])

            ammo = torch.LongTensor([0])
            embs['ammo'] = ammo
            #embs.append(self.embeddings['ammo'](ammo)[0])
        return embs





    def encode_score(self, score):
        score = torch.tensor(score,dtype = torch.float)/16
        return score

    def label_health(self, health):
        health_bins=[0,10,20,30,40,50,65,80,90,100]
        return torch.LongTensor((np.digitize([health],health_bins)))+1

    def label_money(self, money):
        bins = [0,500,1000,1500,2200,3000,3500,3800,4000,4500,5000,5500,7000,9000,12000,16000]
        return torch.LongTensor((np.digitize([money],bins)))+1

    def label_clip(self, clip):
        bins = [0,1,2,10,20,30,35,300]
        return torch.LongTensor((np.digitize([clip],bins)))+1

    def label_ammo(self, ammo):
        bins = [0,2,30,60,120,300]
        return torch.LongTensor((np.digitize([ammo],bins)))+1

    def label_smokes(self, num_smokes):
        bins =[0,1,2,3,4,10]
        return torch.LongTensor((np.digitize([num_smokes],bins)))+1

    def label_molotovs(self, num_molotov):
        bins =[0,1,2,3,4,10]
        return torch.LongTensor((np.digitize([num_molotov],bins)))+1

    def cal_dis(self, pos1, pos2 ):
        pos1 = np.array([pos1['x'],pos1['y'],pos1['z']])
        pos2 = np.array([pos2['x'],pos2['y'],pos2['z']])
        dist = np.linalg.norm(pos1-pos2)
        return dist

    def label_armor(self, armor):
        bins=[0,10,40,80,100]
        return torch.LongTensor((np.digitize([armor],bins)))+1

    def label_has_helmet(self, has_helmet:bool):
        if has_helmet:
            return torch.LongTensor([2])
        else :return torch.LongTensor([1])

    def label_has_defuser(self, has_defuser:bool):
        if has_defuser:
            return torch.LongTensor([2])
        else :return torch.LongTensor([1])


            

In [7]:

class CsData(Dataset):
    def __init__(self, dfx,dfy):
        self.dfx = dfx
        self.dfy = dfy
        self.dfx['map'] = self.dfx['map'].apply(lambda x: maps_label[x])
        self.dfx['round_status'] = self.dfx['round_status'].apply(lambda x: round_status[x])
        self.dfx['round_status_time_left_binned']=self.dfx['round_status_time_left'].apply(lambda x: self.encode_time(x))
        

    def __getitem__(self, id):
        row = self.dfx.iloc[id]
        outs = self.process_row(row)
        #outs = tuple(outs.values())
        return outs, torch.FloatTensor([self.dfy.iloc[id]])

    def __len__(self):
        return len(self.dfx)

    def process_row(self, row):
        
        row_embs = OrderedDict()

        #map
        row_embs['map'] = torch.LongTensor([row['map']])

        #cts and ts
        cts = [p for p in row['alive_players'] if p['team']=='CT']
        ts = [p for p in row['alive_players'] if p['team']=='Terrorist']
        row_embs['cts'] = self.encode_alive_players(cts)
        row_embs['ts'] = self.encode_alive_players(ts)

        # GRS
        
        grs = OrderedDict()
        grs['molotovs'] = self.label_molotovs(len(row['active_molotovs']))
        grs['smokes'] = self.label_smokes(len(row['active_smokes']))
        row_embs['grs'] = grs
        
        #round
        round = OrderedDict()
        round['round_status'] = torch.LongTensor([row['round_status']])
        round['round_status_time_left_binned']= torch.LongTensor([row['round_status_time_left_binned']])
        row_embs['round_info'] = round

        #score
        row_embs['score'] = self.encode_score(row['current_score'])


        return row_embs

    def encode_time(self, time):
        bins = [0, 10, 30, 50, 70, 100,115,4000]
        return (np.digitize([time],bins)[0])+1

    def encode_alive_players(self,alive_pls):
        encoding = []
        for i,plr in enumerate(alive_pls):
            encoding.append(self.player_encoder(plr))

            # bad dta contains more thn 5 in a team
            if i==4:
                break

        for _ in range(len(alive_pls),5):
            encoding.append(self.player_encoder(pad=True))

        return encoding

    def player_encoder(self, plr=None, pad=False):
        embs = OrderedDict()
        if not pad:
            health = self.label_health(plr['health'])
            embs['health_bin'] = health
            #embs.append(self.embeddings['health_bin'](health)[0])
        
            armor= self.label_armor(plr['armor'])
            embs['armor'] = armor
            #embs.append(self.embeddings['armor'](armor)[0])
                        
            money = self.label_money(plr['money'])
            embs['money'] = money
            #embs.append(self.embeddings['money'](money)[0] )
                        
            has_defuser = self.label_has_defuser(plr['has_defuser'])
            embs['has_defuser'] = has_defuser
            #embs.append(self.embeddings['has_defuser'](has_defuser)[0])
                        
            has_helmet = self.label_has_helmet(plr['has_helmet'])
            embs['has_helmet'] = has_helmet
            #embs.append(self.embeddings['has_helmet'](has_helmet)[0])

            embs['inventory']=[]
            for i,item in enumerate(plr['inventory']):
                embs['inventory'].append(self.encode_inventory(item))
                #considering only 6 items
                if i==5:
                    break
            for _ in range(len(plr['inventory']),6):
                embs['inventory'].append(self.encode_inventory(pad=True))

        else:

            health = torch.LongTensor([0])
            embs['health_bin'] = health
            #embs.append(self.embeddings['health_bin'](health)[0])
        
            armor= torch.LongTensor([0])
            embs['armor'] = armor
            #embs.append(self.embeddings['armor'](armor)[0])
                        
            money = torch.LongTensor([0])
            embs['money'] = money
            #embs.append(self.embeddings['money'](money)[0] )
                        
            has_defuser = torch.LongTensor([0])
            embs['has_defuser'] = has_defuser
            #embs.append(self.embeddings['has_defuser'](has_defuser)[0])
                        
            has_helmet = torch.LongTensor([0])
            embs['has_helmet'] = has_helmet
            #embs.append(self.embeddings['has_helmet'](has_helmet)[0])

            embs['inventory']=[]
            for _ in range(6):
                embs['inventory'].append(self.encode_inventory(pad=True))

        return embs


    def encode_inventory(self, item=None, pad = False):
        embs = OrderedDict()
        if not pad:
            typ = item['item_type']
            typ = torch.LongTensor([inventory.index(typ)])
            embs['item'] = typ
            #embs.append(self.embeddings['item'](typ)[0])

            clip = self.label_clip(item['clip_ammo'])
            embs['clip'] = clip
            #embs.append(self.embeddings['clip'](clip)[0])

            ammo = self.label_ammo(item['clip_ammo'])
            embs['ammo'] = ammo
            #embs.append(self.embeddings['ammo'](ammo)[0])

        else: 
            typ = torch.LongTensor([0])
            embs['item'] = typ
            #embs.append(self.embeddings['item'](typ)[0])

            clip = torch.LongTensor([0])
            embs['clip'] = clip
            #embs.append(self.embeddings['clip'](clip)[0])

            ammo = torch.LongTensor([0])
            embs['ammo'] = ammo
            #embs.append(self.embeddings['ammo'](ammo)[0])
        return embs





    def encode_score(self, score):
        score = torch.tensor(score,dtype = torch.float)/16
        return score

    def label_health(self, health):
        health_bins=[0,10,20,30,40,50,65,80,90,100]
        return torch.LongTensor((np.digitize([health],health_bins)))+1

    def label_money(self, money):
        bins = [0,500,1000,1500,2200,3000,3500,3800,4000,4500,5000,5500,7000,9000,12000,16000]
        return torch.LongTensor((np.digitize([money],bins)))+1

    def label_clip(self, clip):
        bins = [0,1,2,10,20,30,35,300]
        return torch.LongTensor((np.digitize([clip],bins)))+1

    def label_ammo(self, ammo):
        bins = [0,2,30,60,120,300]
        return torch.LongTensor((np.digitize([ammo],bins)))+1

    def label_smokes(self, num_smokes):
        bins =[0,1,2,3,4,10]
        return torch.LongTensor((np.digitize([num_smokes],bins)))+1

    def label_molotovs(self, num_molotov):
        bins =[0,1,2,3,4,10]
        return torch.LongTensor((np.digitize([num_molotov],bins)))+1

    def cal_dis(self, pos1, pos2 ):
        pos1 = np.array([pos1['x'],pos1['y'],pos1['z']])
        pos2 = np.array([pos2['x'],pos2['y'],pos2['z']])
        dist = np.linalg.norm(pos1-pos2)
        return dist

    def label_armor(self, armor):
        bins=[0,10,40,80,100]
        return torch.LongTensor((np.digitize([armor],bins)))+1

    def label_has_helmet(self, has_helmet:bool):
        if has_helmet:
            return torch.LongTensor([2])
        else :return torch.LongTensor([1])

    def label_has_defuser(self, has_defuser:bool):
        if has_defuser:
            return torch.LongTensor([2])
        else :return torch.LongTensor([1])


            

In [8]:
class Submodule(nn.Module):
    def __init__(self, hidden1, hidden2, dropout = 0.3):
        super(Submodule, self).__init__()
        self.lnr1 = nn.Linear(hidden1, hidden1)
        self.lnr2 = nn.Linear(hidden1, hidden2)
        self.act = nn.LeakyReLU()
        self.drop = nn.Dropout(p=dropout)
        self.bn = nn.BatchNorm1d(hidden2)

        
    def forward(self, x):
        #print(x.shape)
        x = self.lnr1(x)
        x = self.drop(x)
        x = self.act(x)
        x = self.lnr2(x)
        x = self.bn(x)
        return x

class FC_model(LightningModule):

    def __init__(self,hparams) -> None:
        super(FC_model, self).__init__()
        self.params = hparams
        self.hparams = hparams
        self.embeddings =  nn.ModuleDict()
        self.embeddings['map'] = nn.Embedding(len(maps)+3, 12,padding_idx =0)
        #self.embeddings['map_crc'] = nn.Embedding(25, 2,padding_idx =0)
        #self.embeddings['patch_version'] = nn.Embedding(15,2,padding_idx =0)
        self.embeddings['round_status_time_left_binned'] = nn.Embedding(len(bins['round_time_left'])+3,16,padding_idx =0)
        self.embeddings['round_status'] = nn.Embedding(4+2, 16,padding_idx =0)
        self.embeddings['health_bin'] = nn.Embedding(10+4,16,padding_idx =0)
        self.embeddings['has_helmet'] = nn.Embedding(2+2,16,padding_idx =0)
        self.embeddings['has_defuser'] = nn.Embedding(2+2,4,padding_idx =0)
        #self.embeddings['team'] = nn.Embedding(2+2,12,padding_idx =0)
        self.embeddings['armor'] = nn.Embedding(len(bins['armor'])+2,12,padding_idx =0)
        self.embeddings['item'] = nn.Embedding(len(inventory)+2,12,padding_idx =0)
        self.embeddings['money'] = nn.Embedding(len(bins['money'])+2,16,padding_idx =0)
        self.embeddings['smokes'] = nn.Embedding(len(bins['smokes'])+2,4,padding_idx =0)
        self.embeddings['clip'] = nn.Embedding(len(bins['clip'])+2,4,padding_idx =0)
        self.embeddings['ammo'] = nn.Embedding(len(bins['ammo'])+2,2,padding_idx =0)
        self.embeddings['molotovs'] = nn.Embedding(len(bins['molotov'])+2,4,padding_idx =0)
        

        self.map_nn = Submodule(12,64,0.5)
        self.ct_nn = Submodule(860, 1024,0.3)
        self.t_nn = Submodule(860, 1024,0.3)
        self.gr_nn  = Submodule(8,64,0.8)
        self.round_nn = Submodule(32,64)
        self.score_nn = Submodule(2,64,0.3)

        self.drop = nn.Dropout(p=0.15)
        self.fc1 = Submodule(64*4+1024*2, 3072,0.3)
        self.fc2 = Submodule(3072, 1024,0.3)
        self.classify  = nn.Linear(1024, 1)
        self.loss = nn.BCEWithLogitsLoss()
        self.acc = Accuracy(num_classes=2)

    def forward(self, input):
        input = self.embed_batch(input)
        map, ct,t,gr,round, score = input
        """print(map.shape)
        print(ct.shape)
        print(t.shape)
        print(gr.shape)
        print(round.shape)
        print(score.shape)"""
        map = self.map_nn(map)
        ct = self.ct_nn(ct)
        t = self.t_nn(t)
        gr = self.gr_nn(gr)
        round = self.round_nn(round)
        score = self.score_nn(score)
        

        cat = torch.cat((ct,t,round, map,gr,score), dim =-1)
        cat = self.fc1(cat)
        cat = self.fc2(cat)
        
        out = self.drop(cat)
        output = self.classify(out)

        return output

    def embed_input(self, input):
        i= 0
        map  = self.embeddings[keys[0]](input[0]).squeeze(1)
        i=1
        ct = torch.cat([self.embeddings[i](val).squeeze(1) for i,val in zip(plr_keys*5,input[1:5*len(plr_keys)+1])], dim=-1)
        i+= 5*len(plr_keys)
        ts = torch.cat([self.embeddings[i](val).squeeze(1) for i,val in zip(plr_keys*5,input[i:5*len(plr_keys)+i])], dim=-1)
        i+= 5*len(plr_keys)
        grs = torch.cat([self.embeddings[i](val).squeeze(1) for i,val in zip(gr_keys,input[i:i+len(gr_keys)])], dim=-1)
        i+= len(gr_keys)
        round = torch.cat([self.embeddings[i](val).squeeze(1) for i,val in zip(round_keys,input[i:i+len(round_keys)])], dim=-1)
        i+=len(round_keys)

        return map,ct,ts,grs,round, input[-1]

    def embed_batch(self,batch):
        embs=batch.copy()
        for key, val in batch.items():
            if key=='map':
                
                embs[key]=self.embeddings[key](val).squeeze(1)

            if key == 'grs':
                mol = self.embeddings['molotovs'](val['molotovs']).squeeze(1)
                smoke = self.embeddings['smokes'](val['smokes']).squeeze(1)
                embs[key] = torch.cat((smoke, mol), dim =-1)

            if key == 'round_info':
                round_status_time = self.embeddings['round_status_time_left_binned'](val['round_status_time_left_binned']).squeeze(1)
                round_status= self.embeddings['round_status'](val['round_status']).squeeze(1)
                embs['round_info'] = torch.cat((round_status_time,round_status), dim=-1)

            if key in ('cts' or 'ts'):
                plr_embs=[]
                for pl in val:
                    plr_embs.extend(self.embed_player(pl))
                embs[key] = torch.cat(plr_embs, dim=-1)

        return tuple(embs.values())
            

    def embed_player(self, plr):
        embs = []
        for key, val in plr.items():
            if key=='inventory':
                for item in val:
                    embs.extend(self.embed_inventory(item))
                continue
            embs.append(self.embeddings[key](val).squeeze(1))    

        return embs           

    def embed_inventory(self, item):
        embs = []
        for key, val in item.items():
            embs.append(self.embeddings[key](val).squeeze(1))

        return embs


    def training_step(self, batch, batch_idx):
        input, target  = batch
        #input = self.embed_batch(input)
        pred = self(input)
        
        loss = self.loss(pred, target)
        logs = {'train_loss': loss,}
        """result = pl.TrainResult(minimize=loss,)
        result.prediction = pred
        result.target = target
        result.log('train_loss', loss, prog_bar=True, on_epoch=True)"""
        return {'loss': loss, 'log': logs,"progress_bar": logs, "preds":pred, "targets": target}

    
    def training_epoch_end(self, outputs):
        
        """y_hat = training_step_outputs.prediction
        y_hat = torch.round(torch.sigmoid(y_hat))
        y= training_step_outputs.target"""
        #acc = self.acc(y_hat, y)
        loss = torch.stack([x["loss"] for x in outputs]).mean()
        preds = torch.stack([x["preds"] for x in outputs])
        targets = torch.stack([x["targets"] for x in outputs])
        preds = torch.round(torch.sigmoid(preds))
        #print(preds)
        #print(targets)
        acc = self.acc(preds, targets)
        logs = {'train_loss': loss,'train_acc':acc}
        #training_step_outputs.log('train_acc', acc, on_step = False,prog_bar=True, on_epoch=True)
        #training_step_outputs.acc = acc
        
        return {'loss': loss, 'log': logs,"progress_bar": logs, "acc":acc}


    def validation_step(self, batch, batch_idx):
        input, target  = batch
        #input = self.embed_batch(input)
        pred = self(input)

        loss = self.loss(pred, target)
        logs = {'val_loss': loss,}
        """result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss, prog_bar=True, on_epoch=True)
        result.prediction = pred
        result.target = target"""

        return {'val_loss': loss, 'log': logs,"progress_bar": logs, "preds":pred, "targets": target}
    
        """    
        def validation_end(self, outputs: List[dict]):
        loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        preds = torch.stack([x["preds"] for x in outputs])
        targets = torch.stack([x["targets"] for x in outputs])
        preds = torch.round(torch.sigmoid(preds)
        acc = self.acc(preds, targets)
        return {"val_loss": loss, acc =}"""

    def validation_epoch_end(self, outputs):
        """y_hat = validation_step_outputs.prediction
        y_hat = torch.round(torch.sigmoid(y_hat))
        y= validation_step_outputs.target
        acc = self.acc(y_hat, y)
        
        #validation_step_outputs.log('val_acc', acc, on_step = False,prog_bar=True, on_epoch=True)
        #validation_step_outputs.acc = acc"""
        loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        preds = torch.stack([x["preds"] for x in outputs])
        targets = torch.stack([x["targets"] for x in outputs])
        preds = torch.round(torch.sigmoid(preds))
        acc = self.acc(preds, targets)
        logs = {'val_loss': loss,'val_acc':acc}

        return {"val_loss": loss, 'log': logs,"progress_bar": logs, "val_acc":acc}


    def test_step(self, batch, batch_idx):
        input, target  = batch
        #input = self.embed_batch(input)
        pred = self(input)

        loss = self.loss(pred, target)
        logs = {'test_loss': loss,}
        

        return {'test_loss': loss, 'log': logs,"progress_bar": logs, "preds":pred, "targets": target}
    
        

    def test_end(self, outputs):
        loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        preds = torch.stack([x["preds"] for x in outputs])
        targets = torch.stack([x["targets"] for x in outputs])
        preds = torch.round(torch.sigmoid(preds))
        acc = self.acc(preds, targets)
        logs = {'test_loss': loss,'test_acc':acc}

        return {"test_loss": loss, 'log': logs,"progress_bar": logs, "test_acc":acc}

   
    

    def configure_optimizers(self):
        optims = []
        scheds = []
        no_decay = ['bias','LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in self.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = optim.Adam(optimizer_grouped_parameters, self.params.lr)

        optims.append(optimizer)
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.params.milestones, gamma=0.01)
        
        scheds.append(scheduler)
                
        return optims

    def read_dataset(self, template, start_idx, end_idx):
        frames = [ pd.read_json(f) for f in [template.format(i) for i in range(start_idx, end_idx+1)] ]
        return pd.concat(frames, ignore_index = True)

    def train_dataloader(self):
        dftrain = self.read_dataset(self.params.root+"/dataset_initial/dataset_{:02}.json", 9, 14)
        y_train = dftrain.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)
        dataset = CsData(dftrain, y_train)
        return DataLoader(dataset,
                          batch_size= self.params.batch_size,
                          num_workers=self.params.num_workers,
                          drop_last=True,
                          shuffle = True)
        
    def val_dataloader(self):
        dftrain = self.read_dataset(self.params.root+"/dataset_initial/dataset_{:02}.json", 15, 16)
        y_train = dftrain.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)
        dataset = CsData(dftrain, y_train)
        return DataLoader(dataset,
                          batch_size= self.params.val_batch_size,
                          num_workers=self.params.num_workers,
                          drop_last=True)
        
    def test_dataloader(self):
        dftrain = self.read_dataset(self.params.root+"/dataset_initial/dataset_{:02}.json", 16, 17)
        y_train = dftrain.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)
        dataset = CsData(dftrain, y_train)
        return DataLoader(dataset,
                          batch_size= self.params.val_batch_size,
                          num_workers=self.params.num_workers,
                          drop_last=True)
    


In [9]:
root = '/gdrive/My Drive/dl_projects/CSGOAI'
path = '/gdrive/My Drive/dl_projects/CSGOAI/model3'
path2 = '/gdrive/My Drive/dl_projects/CSGOAI/checkpts'

hparams = Namespace(
        root=root,
        batch_size= 256,
        val_batch_size=256,
        lr= 1e-3,
        epochs=40,
        save_top_k=30,
        accumulate_grad_batches=1,
        num_workers = 4,
        num_warmup_steps=100,
        milestones=[1,3]
      
    )
#wandb_logger = pl_loggers.TensorBoardLogger(path,name = 'lightning_logs',)
wandb_logger = WandbLogger(project='CSGO_AI', save_dir = path)
# initialize Model Checkpoint Saver
checkpoint_callback = ModelCheckpoint(
    filepath=path+'/{epoch}-{val_loss:.3f}-{val_acc:0.3f}-model3',
    save_top_k=hparams.save_top_k,
    verbose=True,
    monitor="val_acc",
    period=1,
    mode="max",
)
model = FC_model(hparams)
trainer = Trainer(gpus=1,default_root_dir = path2, max_epochs=hparams.epochs,logger = wandb_logger,checkpoint_callback=checkpoint_callback,log_gpu_memory=True,weights_summary=None)
#trainer = Trainer(fast_dev_run=True, gpus=1, auto_scale_batch_size= None)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...


Epoch 00036: val_acc  was not in top 30



Epoch 00037: val_acc reached 0.67788 (best 0.69641), saving model to /gdrive/My Drive/dl_projects/CSGOAI/model3/epoch=37-val_loss=0.606-val_acc=0.678-model3.ckpt as top 30



Epoch 00038: val_acc  was not in top 30



Epoch 00039: val_acc reached 0.67378 (best 0.69641), saving model to /gdrive/My Drive/dl_projects/CSGOAI/model3/epoch=39-val_loss=0.642-val_acc=0.674-model3.ckpt as top 30
Saving latest checkpoint..


1

In [ ]:
model=model.load_from_checkpoint('/gdrive/My Drive/dl_projects/CSGOAI/epoch=39-val_model2_loss=0.000.ckpt')

In [ ]:
hparams = Namespace(
        root=root,
        batch_size= 256,
        val_batch_size=256,
        lr= 1e-4,
        epochs=5,
        save_top_k=20,
        accumulate_grad_batches=1,
        num_workers = 4,
        num_warmup_steps=100,
        milestones=[1]
      
    )
model.params=hparams

In [ ]:
trainer.fit(model)

In [ ]:
model=model.load_from_checkpoint('/gdrive/My Drive/dl_projects/CSGOAI/model1_alldata/epoch=3-val_loss=0.526-val_acc=0.702.ckpt')
model.params=hparams
trainer.test(model)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.6978, device='cuda:0'),
 'test_loss': tensor(0.5293, device='cuda:0')}
--------------------------------------------------------------------------------



[{'test_acc': 0.6977983117103577, 'test_loss': 0.529349148273468}]

In [ ]:
print(next(model.parameters()).device)

cuda:0


In [ ]:
model.embeddings['armor'].weight.device

device(type='cpu')

In [ ]:
"""
/gdrive/My Drive/dl_projects/CSGOAI/epoch=39-val_model2_loss=0.000.ckpt' test on 16 17 acc=0.697  
/gdrive/My Drive/dl_projects/CSGOAI/model1_alldata/epoch=7-val_loss=0.529-val_acc=0.700.ckpt' test on 16 17 acc=0.6969 loss0.5319
/gdrive/My Drive/dl_projects/CSGOAI/model1_alldata/epoch=3-val_loss=0.526-val_acc=0.702.ckpt  test on 16 17 acc=0.6977 loss0.5293

"""

In [ ]:
!nvidia-smi

Thu Aug 27 16:53:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:

!pip install pytorch-lightning
!pip install wandb

     |████████████████████████████████| 409kB 5.0MB/s 
     |████████████████████████████████| 276kB 19.4MB/s 
     |████████████████████████████████| 2.8MB 23.1MB/s 
     |████████████████████████████████| 829kB 54.7MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=862d1a2fb56c179eabf7a6f4f3d160283e3d9d6991487de2746cd25c64007832
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=20dfaf37fcb836f5af34d8f2fe8b23d5445f46d83a5c0e87ed70a3c699dc5406
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML future
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.0 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully unin

In [ ]:
pt  = '/gdrive/My Drive/dl_projects/CSGOAI'

In [ ]:
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

import os
from argparse import Namespace
from typing import *
import random
from functools import lru_cache

import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.metrics import Accuracy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torch.nn as nn
import torch
from collections import OrderedDict 
from sklearn.preprocessing import LabelEncoder

In [ ]:

inventory = ['Deagle', 'Elite', 'FiveSeven', 'Glock', 'Ak47', 'Aug', 'Awp', 'Famas', 'G3sg1', 'GalilAr', 'M249', 'M4a4', 'Mac10', 'P90', 'Mp5sd', 'Ump45', 'Xm1014', 'Bizon', 'Mag7', 'Negev', 'Sawedoff', 'Tec9', 'ZeusX27', 'P2000', 'Mp7', 'Mp9', 'Nova', 'P250', 'Scar20', 'Sg553', 'Ssg08', 'M4a1S', 'UspS', 'Cz75Auto', 'R8Revolver', 'Flashbang', 'HeGrenade', 'SmokeGrenade', 'MolotovGrenade', 'DecoyGrenade', 'IncendiaryGrenade', 'Knife', 'C4']
maps = ['de_dust2', 'de_mirage', 'de_nuke', 'de_inferno', 'de_overpass',
       'de_vertigo', 'de_train', 'de_cache']

In [ ]:
maps_label = {map:i+1 for i,map in enumerate(maps)}
round_status = {'BombPlanted':1, 'FreezeTime':2, 'Normal':3, "SlackTime":4}

In [ ]:
bins = {
    'round_time_left':[0, 10, 30, 50, 70, 100,115,4000],
    'health':[0,10,20,30,40,50,65,80,90,100],
    'money':[0,500,100.,1500,2200,3000,3500,3800,4000,4500,5000,5500,7000,9000,12000,16000],
    'clip' : [0,1,2,10,20,30,35,300],
    'ammo' :[0,2,30,60,120,300],
    'smokes':[0,1,2,3,4,10],
    'molotov':[0,1,2,3,4,10],
    'armor':[0,10,40,80,100]
}



In [ ]:
len(bins['armor'])

5

In [ ]:

class CsData(Dataset):
    def __init__(self, dfx,dfy):
        self.dfx = dfx
        self.dfy = dfy
        self.dfx['map'] = self.dfx['map'].apply(lambda x: maps_label[x])
        self.dfx['round_status'] = self.dfx['round_status'].apply(lambda x: round_status[x])
        self.dfx['round_status_time_left_binned']=self.dfx['round_status_time_left'].apply(lambda x: self.encode_time(x))
        

    def __getitem__(self, id):
        row = self.dfx.iloc[id]
        outs = self.process_row(row)
        #outs = tuple(outs.values())
        if len(outs)!= 236:
            print(id)
        return tuple(outs), torch.FloatTensor([self.dfy.iloc[id]])

    def __len__(self):
        return len(self.dfx)

    def process_row(self, row):
        
        row_embs = []#OrderedDict()

        #map
        row_embs.append(torch.LongTensor([row['map']]))

        #cts and ts
        cts = [p for p in row['alive_players'] if p['team']=='CT']
        ts = [p for p in row['alive_players'] if p['team']=='Terrorist']
        row_embs.extend(self.encode_alive_players(cts))
        row_embs.extend(self.encode_alive_players(ts))

        # GRS
        
        grs = OrderedDict()
        grs['molotovs'] = self.label_molotovs(len(row['active_molotovs']))
        grs['smokes'] = self.label_smokes(len(row['active_smokes']))
        row_embs.extend(grs.values())
        
        #round
        round = OrderedDict()
        round['round_status'] = torch.LongTensor([row['round_status']])
        round['round_status_time_left_binned']= torch.LongTensor([row['round_status_time_left_binned']])
        row_embs.extend(round.values())

        #score
        row_embs.append(self.encode_score(row['current_score']))


        return row_embs

    def encode_time(self, time):
        bins = [0, 10, 30, 50, 70, 100,115,4000]
        return (np.digitize([time],bins)[0])+1

    def encode_alive_players(self,alive_pls):
        encoding = []
        for i,plr in enumerate(alive_pls):
            encoding.extend(self.player_encoder(plr))
            # bad dta contains more thn 5 in a team
            if i==4:
                break

        for _ in range(len(alive_pls),5):
            encoding.extend(self.player_encoder(pad=True))

        return encoding

    def player_encoder(self, plr=None, pad=False):
        embs =[]# OrderedDict()
        if not pad:
            health = self.label_health(plr['health'])
            embs.append( health)
            #embs.append(self.embeddings['health_bin'](health)[0])
        
            armor= self.label_armor(plr['armor'])
            embs.append(armor)
            #embs.append(self.embeddings['armor'](armor)[0])
                        
            money = self.label_money(plr['money'])
            embs.append(money)
            #embs.append(self.embeddings['money'](money)[0] )
                        
            has_defuser = self.label_has_defuser(plr['has_defuser'])
            embs.append(has_defuser)
            #embs.append(self.embeddings['has_defuser'](has_defuser)[0])
                        
            has_helmet = self.label_has_helmet(plr['has_helmet'])
            embs.append(has_helmet)
            #embs.append(self.embeddings['has_helmet'](has_helmet)[0])

            #embs['inventory']=[]
            for i,item in enumerate(plr['inventory']):
                embs.extend(self.encode_inventory(item).values())
                # max 6 items considered
                if i==5:
                    break
            for _ in range(len(plr['inventory']),6):
                embs.extend(self.encode_inventory(pad=True).values())

        else:

            health = torch.LongTensor([0])
            embs.append(health)
            #embs.append(self.embeddings['health_bin'](health)[0])
        
            armor= torch.LongTensor([0])
            embs.append(armor)
            #embs.append(self.embeddings['armor'](armor)[0])
                        
            money = torch.LongTensor([0])
            embs.append(money)
            #embs.append(self.embeddings['money'](money)[0] )
                        
            has_defuser = torch.LongTensor([0])
            embs.append(has_defuser)
            #embs.append(self.embeddings['has_defuser'](has_defuser)[0])
                        
            has_helmet = torch.LongTensor([0])
            embs.append(has_helmet)
            #embs.append(self.embeddings['has_helmet'](has_helmet)[0])

            #embs['inventory']=[]
            for _ in range(6):
                embs.extend(self.encode_inventory(pad=True).values())

        return embs


    def encode_inventory(self, item=None, pad = False):
        embs = OrderedDict()
        if not pad:
            typ = item['item_type']
            typ = torch.LongTensor([inventory.index(typ)])
            embs['item'] = typ
            #embs.append(self.embeddings['item'](typ)[0])

            clip = self.label_clip(item['clip_ammo'])
            embs['clip'] = clip
            #embs.append(self.embeddings['clip'](clip)[0])

            ammo = self.label_ammo(item['clip_ammo'])
            embs['ammo'] = ammo
            #embs.append(self.embeddings['ammo'](ammo)[0])

        else: 
            typ = torch.LongTensor([0])
            embs['item'] = typ
            #embs.append(self.embeddings['item'](typ)[0])

            clip = torch.LongTensor([0])
            embs['clip'] = clip
            #embs.append(self.embeddings['clip'](clip)[0])

            ammo = torch.LongTensor([0])
            embs['ammo'] = ammo
            #embs.append(self.embeddings['ammo'](ammo)[0])
        return embs





    def encode_score(self, score):
        score = torch.tensor(score,dtype = torch.float)/16
        return score

    def label_health(self, health):
        health_bins=[0,10,20,30,40,50,65,80,90,100]
        return torch.LongTensor((np.digitize([health],health_bins)))+1

    def label_money(self, money):
        bins = [0,500,1000,1500,2200,3000,3500,3800,4000,4500,5000,5500,7000,9000,12000,16000]
        return torch.LongTensor((np.digitize([money],bins)))+1

    def label_clip(self, clip):
        bins = [0,1,2,10,20,30,35,300]
        return torch.LongTensor((np.digitize([clip],bins)))+1

    def label_ammo(self, ammo):
        bins = [0,2,30,60,120,300]
        return torch.LongTensor((np.digitize([ammo],bins)))+1

    def label_smokes(self, num_smokes):
        bins =[0,1,2,3,4,10]
        return torch.LongTensor((np.digitize([num_smokes],bins)))+1

    def label_molotovs(self, num_molotov):
        bins =[0,1,2,3,4,10]
        return torch.LongTensor((np.digitize([num_molotov],bins)))+1

    def cal_dis(self, pos1, pos2 ):
        pos1 = np.array([pos1['x'],pos1['y'],pos1['z']])
        pos2 = np.array([pos2['x'],pos2['y'],pos2['z']])
        dist = np.linalg.norm(pos1-pos2)
        return dist

    def label_armor(self, armor):
        bins=[0,10,40,80,100]
        return torch.LongTensor((np.digitize([armor],bins)))+1

    def label_has_helmet(self, has_helmet:bool):
        if has_helmet:
            return torch.LongTensor([2])
        else :return torch.LongTensor([1])

    def label_has_defuser(self, has_defuser:bool):
        if has_defuser:
            return torch.LongTensor([2])
        else :return torch.LongTensor([1])


            

In [ ]:

class CsData(Dataset):
    def __init__(self, dfx,dfy):
        self.dfx = dfx
        self.dfy = dfy
        self.dfx['map'] = self.dfx['map'].apply(lambda x: maps_label[x])
        self.dfx['round_status'] = self.dfx['round_status'].apply(lambda x: round_status[x])
        self.dfx['round_status_time_left_binned']=self.dfx['round_status_time_left'].apply(lambda x: self.encode_time(x))
        

    def __getitem__(self, id):
        row = self.dfx.iloc[id]
        outs = self.process_row(row)
        #outs = tuple(outs.values())
        return outs, torch.FloatTensor([self.dfy.iloc[id]])

    def __len__(self):
        return len(self.dfx)

    def process_row(self, row):
        
        row_embs = OrderedDict()

        #map
        row_embs['map'] = torch.LongTensor([row['map']])

        #cts and ts
        cts = [p for p in row['alive_players'] if p['team']=='CT']
        ts = [p for p in row['alive_players'] if p['team']=='Terrorist']
        row_embs['cts'] = self.encode_alive_players(cts)
        row_embs['ts'] = self.encode_alive_players(ts)

        # GRS
        
        grs = OrderedDict()
        grs['molotovs'] = self.label_molotovs(len(row['active_molotovs']))
        grs['smokes'] = self.label_smokes(len(row['active_smokes']))
        row_embs['grs'] = grs
        
        #round
        round = OrderedDict()
        round['round_status'] = torch.LongTensor([row['round_status']])
        round['round_status_time_left_binned']= torch.LongTensor([row['round_status_time_left_binned']])
        row_embs['round_info'] = round

        #score
        row_embs['score'] = self.encode_score(row['current_score'])


        return row_embs

    def encode_time(self, time):
        bins = [0, 10, 30, 50, 70, 100,115,4000]
        return (np.digitize([time],bins)[0])+1

    def encode_alive_players(self,alive_pls):
        encoding = []
        for i,plr in enumerate(alive_pls):
            encoding.append(self.player_encoder(plr))

            # bad dta contains more thn 5 in a team
            if i==4:
                break

        for _ in range(len(alive_pls),5):
            encoding.append(self.player_encoder(pad=True))

        return encoding

    def player_encoder(self, plr=None, pad=False):
        embs = OrderedDict()
        if not pad:
            health = self.label_health(plr['health'])
            embs['health_bin'] = health
            #embs.append(self.embeddings['health_bin'](health)[0])
        
            armor= self.label_armor(plr['armor'])
            embs['armor'] = armor
            #embs.append(self.embeddings['armor'](armor)[0])
                        
            money = self.label_money(plr['money'])
            embs['money'] = money
            #embs.append(self.embeddings['money'](money)[0] )
                        
            has_defuser = self.label_has_defuser(plr['has_defuser'])
            embs['has_defuser'] = has_defuser
            #embs.append(self.embeddings['has_defuser'](has_defuser)[0])
                        
            has_helmet = self.label_has_helmet(plr['has_helmet'])
            embs['has_helmet'] = has_helmet
            #embs.append(self.embeddings['has_helmet'](has_helmet)[0])

            embs['inventory']=[]
            for i,item in enumerate(plr['inventory']):
                embs['inventory'].append(self.encode_inventory(item))
                #considering only 6 items
                if i==5:
                    break
            for _ in range(len(plr['inventory']),6):
                embs['inventory'].append(self.encode_inventory(pad=True))

        else:

            health = torch.LongTensor([0])
            embs['health_bin'] = health
            #embs.append(self.embeddings['health_bin'](health)[0])
        
            armor= torch.LongTensor([0])
            embs['armor'] = armor
            #embs.append(self.embeddings['armor'](armor)[0])
                        
            money = torch.LongTensor([0])
            embs['money'] = money
            #embs.append(self.embeddings['money'](money)[0] )
                        
            has_defuser = torch.LongTensor([0])
            embs['has_defuser'] = has_defuser
            #embs.append(self.embeddings['has_defuser'](has_defuser)[0])
                        
            has_helmet = torch.LongTensor([0])
            embs['has_helmet'] = has_helmet
            #embs.append(self.embeddings['has_helmet'](has_helmet)[0])

            embs['inventory']=[]
            for _ in range(6):
                embs['inventory'].append(self.encode_inventory(pad=True))

        return embs


    def encode_inventory(self, item=None, pad = False):
        embs = OrderedDict()
        if not pad:
            typ = item['item_type']
            typ = torch.LongTensor([inventory.index(typ)])
            embs['item'] = typ
            #embs.append(self.embeddings['item'](typ)[0])

            clip = self.label_clip(item['clip_ammo'])
            embs['clip'] = clip
            #embs.append(self.embeddings['clip'](clip)[0])

            ammo = self.label_ammo(item['clip_ammo'])
            embs['ammo'] = ammo
            #embs.append(self.embeddings['ammo'](ammo)[0])

        else: 
            typ = torch.LongTensor([0])
            embs['item'] = typ
            #embs.append(self.embeddings['item'](typ)[0])

            clip = torch.LongTensor([0])
            embs['clip'] = clip
            #embs.append(self.embeddings['clip'](clip)[0])

            ammo = torch.LongTensor([0])
            embs['ammo'] = ammo
            #embs.append(self.embeddings['ammo'](ammo)[0])
        return embs





    def encode_score(self, score):
        score = torch.tensor(score,dtype = torch.float)/16
        return score

    def label_health(self, health):
        health_bins=[0,10,20,30,40,50,65,80,90,100]
        return torch.LongTensor((np.digitize([health],health_bins)))+1

    def label_money(self, money):
        bins = [0,500,1000,1500,2200,3000,3500,3800,4000,4500,5000,5500,7000,9000,12000,16000]
        return torch.LongTensor((np.digitize([money],bins)))+1

    def label_clip(self, clip):
        bins = [0,1,2,10,20,30,35,300]
        return torch.LongTensor((np.digitize([clip],bins)))+1

    def label_ammo(self, ammo):
        bins = [0,2,30,60,120,300]
        return torch.LongTensor((np.digitize([ammo],bins)))+1

    def label_smokes(self, num_smokes):
        bins =[0,1,2,3,4,10]
        return torch.LongTensor((np.digitize([num_smokes],bins)))+1

    def label_molotovs(self, num_molotov):
        bins =[0,1,2,3,4,10]
        return torch.LongTensor((np.digitize([num_molotov],bins)))+1

    def cal_dis(self, pos1, pos2 ):
        pos1 = np.array([pos1['x'],pos1['y'],pos1['z']])
        pos2 = np.array([pos2['x'],pos2['y'],pos2['z']])
        dist = np.linalg.norm(pos1-pos2)
        return dist

    def label_armor(self, armor):
        bins=[0,10,40,80,100]
        return torch.LongTensor((np.digitize([armor],bins)))+1

    def label_has_helmet(self, has_helmet:bool):
        if has_helmet:
            return torch.LongTensor([2])
        else :return torch.LongTensor([1])

    def label_has_defuser(self, has_defuser:bool):
        if has_defuser:
            return torch.LongTensor([2])
        else :return torch.LongTensor([1])


            

In [ ]:
x = dftrain.copy()
y= y_train.copy()
#x = dfeval.copy()
#y= y_eval.copy()
data = CsData(x,y)

In [ ]:

(data[430])

In [ ]:
(x.iloc[43010]['alive_players'])

In [ ]:
"""

"""

In [ ]:
dl=DataLoader(data, shuffle=True,
                          batch_size= 120,
                          num_workers=1,
                          drop_last=True)


In [ ]:
for batch_idx, batch in enumerate(dl):
    continue

In [ ]:
item_keys = ['item', 'clip', 'ammo']
plr_keys = ['health_bin', 'armor', 'money', 'has_defuser', 'has_helmet'] + item_keys*6
gr_keys = ['molotovs', 'smokes']
round_keys = ['round_status', 'round_status_time_left_binned']
keys = ['map'] + plr_keys*10 + gr_keys + round_keys

In [ ]:
len(keys)

235

In [ ]:
pt  = '/gdrive/My Drive/dl_projects/CSGOAI'
def read_dataset(template, start_idx, end_idx):
    frames = [ pd.read_json(f) for f in [template.format(i) for i in range(start_idx, end_idx+1)] ]
    return pd.concat(frames, ignore_index = True)

"""dftrain = read_dataset(pt+"/dataset_initial/dataset_{:02}.json", 9, 14)
y_train = dftrain.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)"""

dfeval = read_dataset(pt+"/dataset_initial/dataset_{:02}.json", 15, 16)
y_eval = dfeval.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)

In [ ]:
x

In [ ]:
class Submodule(nn.Module):
    def __init__(self, hidden1, hidden2, dropout = 0.3):
        super(Submodule, self).__init__()
        self.lnr1 = nn.Linear(hidden1, hidden1)
        self.lnr2 = nn.Linear(hidden1, hidden2)
        self.act = nn.LeakyReLU()
        self.drop = nn.Dropout(p=dropout)
        self.bn = nn.BatchNorm1d(hidden2)

        
    def forward(self, x):
        #print(x.shape)
        x = self.lnr1(x)
        x = self.drop(x)
        x = self.act(x)
        x = self.lnr2(x)
        x = self.bn(x)
        return x

class FC_model(LightningModule):

    def __init__(self,hparams) -> None:
        super(FC_model, self).__init__()
        self.params = hparams
        self.hparams = hparams
        self.embeddings =  nn.ModuleDict()
        self.embeddings['map'] = nn.Embedding(len(maps)+3, 12,padding_idx =0)
        #self.embeddings['map_crc'] = nn.Embedding(25, 2,padding_idx =0)
        #self.embeddings['patch_version'] = nn.Embedding(15,2,padding_idx =0)
        self.embeddings['round_status_time_left_binned'] = nn.Embedding(len(bins['round_time_left'])+3,24,padding_idx =0)
        self.embeddings['round_status'] = nn.Embedding(4+2, 24,padding_idx =0)
        self.embeddings['health_bin'] = nn.Embedding(10+4,24,padding_idx =0)
        self.embeddings['has_helmet'] = nn.Embedding(2+2,12,padding_idx =0)
        self.embeddings['has_defuser'] = nn.Embedding(2+2,4,padding_idx =0)
        #self.embeddings['team'] = nn.Embedding(2+2,12,padding_idx =0)
        self.embeddings['armor'] = nn.Embedding(len(bins['armor'])+2,12,padding_idx =0)
        self.embeddings['item'] = nn.Embedding(len(inventory)+2,12,padding_idx =0)
        self.embeddings['money'] = nn.Embedding(len(bins['money'])+2,24,padding_idx =0)
        self.embeddings['smokes'] = nn.Embedding(len(bins['smokes'])+2,4,padding_idx =0)
        self.embeddings['clip'] = nn.Embedding(len(bins['clip'])+2,4,padding_idx =0)
        self.embeddings['ammo'] = nn.Embedding(len(bins['ammo'])+2,2,padding_idx =0)
        self.embeddings['molotovs'] = nn.Embedding(len(bins['molotov'])+2,4,padding_idx =0)
        

        self.map_nn = Submodule(12,64,0.5)
        self.ct_nn = Submodule(920, 1024,0.3)
        self.t_nn = Submodule(920, 1024,0.3)
        self.gr_nn  = Submodule(8,128,0.8)
        self.round_nn = Submodule(48,128)
        self.score_nn = Submodule(2,128,0.3)

        self.drop = nn.Dropout(p=0.1)
        self.fc1 = Submodule(128*3+1024*2+64, 3072,0.3)
        self.fc2 = Submodule(3072, 1024,0.2)
        self.classify  = nn.Linear(1024, 1)
        self.loss = nn.BCEWithLogitsLoss()
        self.acc = Accuracy(num_classes=2)

    def forward(self, input):
        input = self.embed_batch(input)
        map, ct,t,gr,round, score = input
        """print(map.shape)
        print(ct.shape)
        print(t.shape)
        print(gr.shape)
        print(round.shape)
        print(score.shape)"""
        map = self.map_nn(map)
        ct = self.ct_nn(ct)
        t = self.t_nn(t)
        gr = self.gr_nn(gr)
        round = self.round_nn(round)
        score = self.score_nn(score)
        

        cat = torch.cat((ct,t,round, map,gr,score), dim =-1)
        cat = self.fc1(cat)
        cat = self.fc2(cat)
        
        out = self.drop(cat)
        output = self.classify(out)

        return output

    def embed_input(self, input):
        i= 0
        map  = self.embeddings[keys[0]](input[0]).squeeze(1)
        i=1
        ct = torch.cat([self.embeddings[i](val).squeeze(1) for i,val in zip(plr_keys*5,input[1:5*len(plr_keys)+1])], dim=-1)
        i+= 5*len(plr_keys)
        ts = torch.cat([self.embeddings[i](val).squeeze(1) for i,val in zip(plr_keys*5,input[i:5*len(plr_keys)+i])], dim=-1)
        i+= 5*len(plr_keys)
        grs = torch.cat([self.embeddings[i](val).squeeze(1) for i,val in zip(gr_keys,input[i:i+len(gr_keys)])], dim=-1)
        i+= len(gr_keys)
        round = torch.cat([self.embeddings[i](val).squeeze(1) for i,val in zip(round_keys,input[i:i+len(round_keys)])], dim=-1)
        i+=len(round_keys)

        return map,ct,ts,grs,round, input[-1]

    def embed_batch(self,batch):
        embs=batch.copy()
        for key, val in batch.items():
            if key=='map':
                
                embs[key]=self.embeddings[key](val).squeeze(1)

            if key == 'grs':
                mol = self.embeddings['molotovs'](val['molotovs']).squeeze(1)
                smoke = self.embeddings['smokes'](val['smokes']).squeeze(1)
                embs[key] = torch.cat((smoke, mol), dim =-1)

            if key == 'round_info':
                round_status_time = self.embeddings['round_status_time_left_binned'](val['round_status_time_left_binned']).squeeze(1)
                round_status= self.embeddings['round_status'](val['round_status']).squeeze(1)
                embs['round_info'] = torch.cat((round_status_time,round_status), dim=-1)

            if key in ('cts' or 'ts'):
                plr_embs=[]
                for pl in val:
                    plr_embs.extend(self.embed_player(pl))
                embs[key] = torch.cat(plr_embs, dim=-1)

        return tuple(embs.values())
            

    def embed_player(self, plr):
        embs = []
        for key, val in plr.items():
            if key=='inventory':
                for item in val:
                    embs.extend(self.embed_inventory(item))
                continue
            embs.append(self.embeddings[key](val).squeeze(1))    

        return embs           

    def embed_inventory(self, item):
        embs = []
        for key, val in item.items():
            embs.append(self.embeddings[key](val).squeeze(1))

        return embs


    def training_step(self, batch, batch_idx):
        input, target  = batch
        #input = self.embed_batch(input)
        pred = self(input)
        
        loss = self.loss(pred, target)
        logs = {'train_loss': loss,}
        """result = pl.TrainResult(minimize=loss,)
        result.prediction = pred
        result.target = target
        result.log('train_loss', loss, prog_bar=True, on_epoch=True)"""
        return {'loss': loss, 'log': logs,"progress_bar": logs, "preds":pred, "targets": target}

    
    def training_epoch_end(self, outputs):
        
        """y_hat = training_step_outputs.prediction
        y_hat = torch.round(torch.sigmoid(y_hat))
        y= training_step_outputs.target"""
        #acc = self.acc(y_hat, y)
        loss = torch.stack([x["loss"] for x in outputs]).mean()
        preds = torch.stack([x["preds"] for x in outputs])
        targets = torch.stack([x["targets"] for x in outputs])
        preds = torch.round(torch.sigmoid(preds))
        #print(preds)
        #print(targets)
        acc = self.acc(preds, targets)
        logs = {'train_loss': loss,'train_acc':acc}
        #training_step_outputs.log('train_acc', acc, on_step = False,prog_bar=True, on_epoch=True)
        #training_step_outputs.acc = acc
        
        return {'loss': loss, 'log': logs,"progress_bar": logs, "acc":acc}


    def validation_step(self, batch, batch_idx):
        input, target  = batch
        #input = self.embed_batch(input)
        pred = self(input)

        loss = self.loss(pred, target)
        logs = {'val_loss': loss,}
        """result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss, prog_bar=True, on_epoch=True)
        result.prediction = pred
        result.target = target"""

        return {'val_loss': loss, 'log': logs,"progress_bar": logs, "preds":pred, "targets": target}
    
        """    
        def validation_end(self, outputs: List[dict]):
        loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        preds = torch.stack([x["preds"] for x in outputs])
        targets = torch.stack([x["targets"] for x in outputs])
        preds = torch.round(torch.sigmoid(preds)
        acc = self.acc(preds, targets)
        return {"val_loss": loss, acc =}"""

    def validation_epoch_end(self, outputs):
        """y_hat = validation_step_outputs.prediction
        y_hat = torch.round(torch.sigmoid(y_hat))
        y= validation_step_outputs.target
        acc = self.acc(y_hat, y)
        
        #validation_step_outputs.log('val_acc', acc, on_step = False,prog_bar=True, on_epoch=True)
        #validation_step_outputs.acc = acc"""
        loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        preds = torch.stack([x["preds"] for x in outputs])
        targets = torch.stack([x["targets"] for x in outputs])
        preds = torch.round(torch.sigmoid(preds))
        acc = self.acc(preds, targets)
        logs = {'val_loss': loss,'val_acc':acc}

        return {"val_loss": loss, 'log': logs,"progress_bar": logs, "val_acc":acc}


    def test_step(self, batch, batch_idx):
        input, target  = batch
        #input = self.embed_batch(input)
        pred = self(input)

        loss = self.loss(pred, target)
        logs = {'test_loss': loss,}
        

        return {'test_loss': loss, 'log': logs,"progress_bar": logs, "preds":pred, "targets": target}
    
        

    def test_end(self, outputs):
        loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        preds = torch.stack([x["preds"] for x in outputs])
        targets = torch.stack([x["targets"] for x in outputs])
        preds = torch.round(torch.sigmoid(preds))
        acc = self.acc(preds, targets)
        logs = {'test_loss': loss,'test_acc':acc}

        return {"test_loss": loss, 'log': logs,"progress_bar": logs, "test_acc":acc}

   
    

    def configure_optimizers(self):
        optims = []
        scheds = []
        no_decay = ['bias','LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in self.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = optim.Adam(optimizer_grouped_parameters, self.params.lr)

        optims.append(optimizer)
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.params.milestones, gamma=0.01)
        
        scheds.append(scheduler)
                
        return optims

    def read_dataset(self, template, start_idx, end_idx):
        frames = [ pd.read_json(f) for f in [template.format(i) for i in range(start_idx, end_idx+1)] ]
        return pd.concat(frames, ignore_index = True)

    def train_dataloader(self):
        dftrain = self.read_dataset(self.params.root+"/dataset_initial/dataset_{:02}.json", 9, 14)
        y_train = dftrain.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)
        dataset = CsData(dftrain, y_train)
        return DataLoader(dataset,
                          batch_size= self.params.batch_size,
                          num_workers=self.params.num_workers,
                          drop_last=True,
                          shuffle = True)
        
    def val_dataloader(self):
        dftrain = self.read_dataset(self.params.root+"/dataset_initial/dataset_{:02}.json", 15, 16)
        y_train = dftrain.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)
        dataset = CsData(dftrain, y_train)
        return DataLoader(dataset,
                          batch_size= self.params.val_batch_size,
                          num_workers=self.params.num_workers,
                          drop_last=True)
        
    def test_dataloader(self):
        dftrain = self.read_dataset(self.params.root+"/dataset_initial/dataset_{:02}.json", 16, 17)
        y_train = dftrain.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)
        dataset = CsData(dftrain, y_train)
        return DataLoader(dataset,
                          batch_size= self.params.val_batch_size,
                          num_workers=self.params.num_workers,
                          drop_last=True)
    


In [ ]:
root = '/gdrive/My Drive/dl_projects/CSGOAI'
path = '/gdrive/My Drive/dl_projects/CSGOAI/model1_alldata'
path2 = '/gdrive/My Drive/dl_projects/CSGOAI/checkpts'

hparams = Namespace(
        root=root,
        batch_size= 256,
        val_batch_size=256,
        lr= 1e-3,
        epochs=40,
        save_top_k=30,
        accumulate_grad_batches=1,
        num_workers = 4,
        num_warmup_steps=100,
        milestones=[1,3]
      
    )
#wandb_logger = pl_loggers.TensorBoardLogger(path,name = 'lightning_logs',)
wandb_logger = WandbLogger(project='CSGO_AI', save_dir = path)
# initialize Model Checkpoint Saver
checkpoint_callback = ModelCheckpoint(
    filepath=path+'/{epoch}-{val_loss:.3f}-{val_acc:0.3f}',
    save_top_k=hparams.save_top_k,
    verbose=True,
    monitor="val_acc",
    period=1,
    mode="max",
)
model = FC_model(hparams)
trainer = Trainer(gpus=1,default_root_dir = path2, max_epochs=hparams.epochs,logger = wandb_logger,checkpoint_callback=checkpoint_callback,log_gpu_memory=True,weights_summary=None)
#trainer = Trainer(fast_dev_run=True, gpus=1, auto_scale_batch_size= None)
#trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [ ]:
model=model.load_from_checkpoint('/gdrive/My Drive/dl_projects/CSGOAI/epoch=39-val_model2_loss=0.000.ckpt')

In [ ]:
hparams = Namespace(
        root=root,
        batch_size= 256,
        val_batch_size=256,
        lr= 1e-4,
        epochs=5,
        save_top_k=20,
        accumulate_grad_batches=1,
        num_workers = 4,
        num_warmup_steps=100,
        milestones=[1]
      
    )
model.params=hparams

In [ ]:
trainer.fit(model)

In [ ]:
model=model.load_from_checkpoint('/gdrive/My Drive/dl_projects/CSGOAI/model1_alldata/epoch=3-val_loss=0.526-val_acc=0.702.ckpt')
model.params=hparams
trainer.test(model)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.6978, device='cuda:0'),
 'test_loss': tensor(0.5293, device='cuda:0')}
--------------------------------------------------------------------------------



[{'test_acc': 0.6977983117103577, 'test_loss': 0.529349148273468}]

In [ ]:
print(next(model.parameters()).device)

cuda:0


In [ ]:
model.embeddings['armor'].weight.device

device(type='cpu')

In [ ]:
"""
/gdrive/My Drive/dl_projects/CSGOAI/epoch=39-val_model2_loss=0.000.ckpt' test on 16 17 acc=0.697  
/gdrive/My Drive/dl_projects/CSGOAI/model1_alldata/epoch=7-val_loss=0.529-val_acc=0.700.ckpt' test on 16 17 acc=0.6969 loss0.5319
/gdrive/My Drive/dl_projects/CSGOAI/model1_alldata/epoch=3-val_loss=0.526-val_acc=0.702.ckpt  test on 16 17 acc=0.6977 loss0.5293

"""